In [ ]:
from src.covid import get_data, plot_history, fi_tz
import datetime

## Koko Suomen koronatilanne

In [ ]:
all_data = get_data("Kaikki sairaanhoitopiirit")
all_data.iloc[-10:]

In [ ]:
plot_history("Kaikki sairaanhoitopiirit")

## Uudenmaan koronatilanne

### Epidemian alusta

In [ ]:
plot_history("HUS")

In [ ]:
hus_data = get_data("HUS")
hus_data.iloc[-10:]

### Viime ajat

In [ ]:
plot_history("HUS", start_date=datetime.datetime(2021,1,1, tzinfo=fi_tz))
